<a href="https://colab.research.google.com/github/imihawk77/Imihawk77/blob/main/Hw_14_%D1%82%D0%B5%D0%BB%D0%B5%D0%B3%D1%80%D0%B0%D0%BC_%D0%B1%D0%BE%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.2/698.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.1
    Uninstalling aiohttp-3.13.1:
      Successfully uninstalled aiohttp-3.13.1


In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install openai

In [ ]:
!pip install aiosqlite

In [ ]:
import os
from getpass import getpass


# Введите ключи безопасно
BOT_TOKEN = os.getenv("API_TOKEN") or getpass("Введите API_TOKEN: ").strip()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass("Введите OPENAI_API_KEY: ").strip()

# Модели
EMBEDDING_MODEL = "text-embedding-3-small"  # для эмбеддингов
GEN_MODEL = "gpt-4o-mini"                   # для генерации ответа

# Параметры RAG
TOP_K = 4
MAX_CONTEXT_CHARS = 9000

Введите API_TOKEN: ··········
Введите OPENAI_API_KEY: ··········


In [ ]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

Saving ocean.csv to ocean.csv
dict_keys(['ocean.csv'])


In [ ]:
import io
import asyncio
import pandas as pd
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.methods import DeleteWebhook
from aiogram.types import Message
from openai import OpenAI


logging.basicConfig(level=logging.INFO)
bot = Bot(BOT_TOKEN)
dp = Dispatcher()
df = pd.read_csv(io.BytesIO(uploaded['ocean.csv']))

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    await message.answer('Привет! Я телеграм бот с ChatGPT!', parse_mode='HTML')

@dp.message(Command("help"))
async def on_help(message: Message):
    example = "what is the deepest ocean."
    await message.answer(
        "ℹ️ База знаний:\n"
        f"• Тематика: Ocean\n"
        f"• Число записей: {len(df)}\n"
        f"• Пример запроса: {example}"
    )

@dp.message(lambda message: message.text)
async def filter_message(message: Message):
    client = OpenAI(
        api_key=OPENAI_API_KEY
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": message.text}
        ]
    )
    text = completion.choices[0].message.content
    await message.answer(text, parse_mode="Markdown")

async def main():
    await bot(DeleteWebhook(drop_pending_updates=True))
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())